In [1]:
import cv2
import numpy as np
import os
import random
import shutil


In [2]:
def stage1_basic_transforms(image_path, output_dir):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image from {image_path}")

    os.makedirs(output_dir, exist_ok=True)

    h, w = img.shape[:2]

    # 1. Zoom In
    zoom_in = img[h//4:3*h//4, w//4:3*w//4]
    zoom_in = cv2.resize(zoom_in, (w, h))
    cv2.imwrite(os.path.join(output_dir, "zoom_in.jpg"), zoom_in)

    # 2. Zoom Out
    zoom_out = cv2.resize(img, (w//2, h//2))
    zoom_out = cv2.copyMakeBorder(zoom_out, h//4, h//4, w//4, w//4,
                                  cv2.BORDER_CONSTANT, value=(0,0,0))
    cv2.imwrite(os.path.join(output_dir, "zoom_out.jpg"), zoom_out)

    # 3. Rotation
    M = cv2.getRotationMatrix2D((w/2, h/2), 15, 1)
    rotated = cv2.warpAffine(img, M, (w, h))
    cv2.imwrite(os.path.join(output_dir, "rotated.jpg"), rotated)

    # 4. Tilt
    pts1 = np.float32([[0,0], [w-1,0], [0,h-1]])
    pts2 = np.float32([[0,0], [w-1, int(0.3*h)], [int(0.2*w), h-1]])
    M_tilt = cv2.getAffineTransform(pts1, pts2)
    tilted = cv2.warpAffine(img, M_tilt, (w, h))
    cv2.imwrite(os.path.join(output_dir, "tilted.jpg"), tilted)

    # 5. Flip
    flipped = cv2.flip(img, 1)
    cv2.imwrite(os.path.join(output_dir, "flipped.jpg"), flipped)

    # 6. Blur
    blurred = cv2.GaussianBlur(img, (9, 9), 0)
    cv2.imwrite(os.path.join(output_dir, "blurred.jpg"), blurred)

    # 7. Brightness
    bright = cv2.convertScaleAbs(img, alpha=1.2, beta=30)
    cv2.imwrite(os.path.join(output_dir, "bright.jpg"), bright)

    # 8. Translation
    M_trans = np.float32([[1, 0, 30], [0, 1, 50]])
    translated = cv2.warpAffine(img, M_trans, (w, h))
    cv2.imwrite(os.path.join(output_dir, "translated.jpg"), translated)



In [3]:

def apply_transformations(image_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image from {image_path}")

    h, w = image.shape[:2]

    # 1. Skewing Horizontal
    skew_matrix_h = np.array([[1, 0.3, 0], [0, 1, 0]], dtype=np.float32)
    skewed_image_h = cv2.warpAffine(image, skew_matrix_h, (int(w * 1.3), h))
    cv2.imwrite(os.path.join(output_dir, "skewed_horizontal.jpg"), skewed_image_h)

    # 2. Perspective
    pts1 = np.float32([[50, 50], [w - 50, 50], [50, h - 50], [w - 50, h - 50]])
    pts2 = np.float32([[0, 0], [w, 0], [100, h], [w - 100, h]])
    perspective_matrix = cv2.getPerspectiveTransform(pts1, pts2)
    perspective_image = cv2.warpPerspective(image, perspective_matrix, (w, h))
    cv2.imwrite(os.path.join(output_dir, "perspective.jpg"), perspective_image)

    # 3. Shear
    shear_matrix = np.float32([[1, 0.5, 0], [0, 1, 0]])
    sheared_image = cv2.warpAffine(image, shear_matrix, (int(w + h * 0.5), h))
    cv2.imwrite(os.path.join(output_dir, "sheared.jpg"), sheared_image)

    # 4. Stretch
    stretched_image = cv2.resize(image, (int(w * 1.5), h))
    cv2.imwrite(os.path.join(output_dir, "stretched.jpg"), stretched_image)

    # 5. Compression
    compressed_image = cv2.resize(image, (int(w * 0.5), h))
    cv2.imwrite(os.path.join(output_dir, "compressed.jpg"), compressed_image)



In [4]:

def process_dataset(dataset_dir, num_images=10, output_root="outputs"):
    # Ensure outputs dir exists
    os.makedirs(output_root, exist_ok=True)

    # Select images
    all_images = [f for f in os.listdir(dataset_dir)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if len(all_images) < num_images:
        raise ValueError("Not enough images in dataset to sample from.")

    selected = random.sample(all_images, num_images)

    # Save selected images
    selected_dir = os.path.join(output_root, "selected")
    os.makedirs(selected_dir, exist_ok=True)

    selected_paths = []
    for img_name in selected:
        src_path = os.path.join(dataset_dir, img_name)
        dst_path = os.path.join(selected_dir, img_name)
        shutil.copy(src_path, dst_path)
        selected_paths.append(dst_path)

    print(f"Selected {num_images} images and saved in {selected_dir}")

    # Apply transformations
    for idx, img_path in enumerate(selected_paths):
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        # Stage1 outputs
        stage1_dir = os.path.join(output_root, "stage1", base_name)
        stage1_basic_transforms(img_path, stage1_dir)

        # Apply transformations outputs
        stage2_dir = os.path.join(output_root, "stage2", base_name)
        apply_transformations(img_path, stage2_dir)

    print(f"All transformations completed and saved under {output_root}")


In [6]:

dataset_dir = "../Datasets/images"
process_dataset(dataset_dir, num_images=10, output_root="outputs")


Selected 10 images and saved in outputs\selected
All transformations completed and saved under outputs
